In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!unzip "/content/gdrive/MyDrive/angiography_dataset/train.zip" -d "/content/"
!unzip "/content/gdrive/MyDrive/angiography_dataset/test.zip" -d "/content/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: /content/train_set/1522RV-B.jpg  
 extracting: /content/train_set/1523LI-A.jpg  
 extracting: /content/train_set/1523LI-B.jpg  
 extracting: /content/train_set/1523LV-A.jpg  
 extracting: /content/train_set/1523LV-B.jpg  
 extracting: /content/train_set/1523RI-A.jpg  
 extracting: /content/train_set/1523RI-B.jpg  
 extracting: /content/train_set/1523RV-A.jpg  
 extracting: /content/train_set/1523RV-B.jpg  
 extracting: /content/train_set/1524LI-A.jpg  
 extracting: /content/train_set/1524LI-B.jpg  
 extracting: /content/train_set/1524LV-A.jpg  
 extracting: /content/train_set/1524LV-B.jpg  
 extracting: /content/train_set/1524RI-A.jpg  
 extracting: /content/train_set/1524RI-B.jpg  
 extracting: /content/train_set/1524RV-A.jpg  
 extracting: /content/train_set/1524RV-B.jpg  
 extracting: /content/train_set/1525LI-A.jpg  
 extracting: /content/train_set/1525LI-B.jpg  
 extracting: /content/train_set/1525LV-A.jpg  
 extracting: /content/tr

CNN

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
import pandas as pd
import os
import glob


class AneurysmDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform=transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get current index (patient ID)
        patient_id = self.df.iloc[idx, 0]

        # Open only the 8 images corresponding to the current patient ID
        image_files = [f"{patient_id}{suffix}.jpg" for suffix in ['LI-A', 'LI-B', 'RI-A', 'RI-B', 'LV-A', 'LV-B', 'RV-A', 'RV-B']]
        images = [Image.open(os.path.join(self.img_dir, img_name)) for img_name in image_files]

        if self.transform:
            images = torch.cat([self.transform(image) for image in images], dim=0)

        # labels_location = self.df.iloc[idx, -21:].astype('float')
        # labels_location = torch.tensor(labels_location).float()

        labels=self.df.iloc[idx,2:].astype('float')
        labels = torch.tensor(labels).float()


        return patient_id, images, labels


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(8, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64*32*32, 64)  # Assumes image size is 128*128

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn = CNN()
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 21)  # 21 labels to predict

    def forward(self, image):
        x = self.cnn(image)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


Resnet + LSTM

In [34]:
import torch
from torch import nn
from torchvision.models import resnet50,ResNet50_Weights
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
import pandas as pd
import os
import glob


class AneurysmDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform=transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()

      # Get current index (patient ID)
      patient_id = self.df.iloc[idx, 0]

      # Open only the 8 images corresponding to the current patient ID
      image_files = [f"{patient_id}{suffix}.jpg" for suffix in ['LI-A', 'LI-B', 'RI-A', 'RI-B', 'LV-A', 'LV-B', 'RV-A', 'RV-B']]
      images = [Image.open(os.path.join(self.img_dir, img_name)) for img_name in image_files]

      if self.transform:
          # Apply transform to each image and add sequence dimension
          images = [self.transform(image).unsqueeze(0) for image in images]
          # Concatenate along the sequence dimension
          images = torch.cat(images, dim=0)

      labels=self.df.iloc[idx,2:].astype('float')
      labels = torch.tensor(labels).float()

      return patient_id, images, labels


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Pretrained ResNet50 without the last FC layer
        self.resnet = nn.Sequential(*list(resnet50(weights=ResNet50_Weights.IMAGENET1K_V2).children())[:-1])
        self.rnn = nn.LSTM(input_size=2048, hidden_size=64, batch_first=True)  # Adjusted input size according to ResNet50 output
        self.fc = nn.Linear(64, 21)  # 21 labels to predict

    def forward(self, images):
        batch_size, seq_length, c, h, w = images.size()
        # Reshape images to (-1, c, h, w) to fit ResNet
        images = images.view(-1, c, h, w)
        # Extract features using ResNet
        features = self.resnet(images)
        # Reshape back to (batch_size, seq_length, -1)
        features = features.view(batch_size, seq_length, -1)
        # Run LSTM
        out, _ = self.rnn(features)
        # Only use the last output of the LSTM sequence for classification
        out = out[:, -1, :]
        out = torch.sigmoid(self.fc(out))
        return out


In [35]:
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torch.utils.data.dataset import random_split


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop((128, 128), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(degrees=20),
    # transforms.Grayscale(),
    transforms.ToTensor()
])

# Define a transform function for validation and test data
val_test_transform = transforms.Compose([
    # transforms.Grayscale(),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])


dataset = AneurysmDataset('/content/train_set/train.csv', '/content/train_set/', transform=train_transform)
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=0)

# Define the lengths of your splits. Here, let's say 80% for training, 10% for validation, and 10% for testing
train_len = int(len(dataset)*0.8)
val_len = int(len(dataset)*0.1)
test_len = len(dataset) - train_len - val_len

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_len, val_len, test_len])

# Apply the validation/test transformations to the validation and test datasets
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

In [46]:
# Instantiate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
n_epochs=10

model = Net().to(device)  # 'device' is either a GPU or CPU

# Define the loss and optimizer
criterion_bce = nn.BCELoss()  # Use Binary Cross Entropy Loss for multi-label classification
criterion_mse = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Path to the saved model
checkpoint_path = "/content/model_epoch_9.pth"  # Replace this with your path

# Load the model weights
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f"Loaded checkpoint: {checkpoint_path}")
else:
    print("No checkpoint found at: "+checkpoint_path)

# Training loop
for epoch in range(10,10+n_epochs):
    model.train()
    for i, (patient_ids, images, labels) in enumerate(train_loader):  # Assuming 'train_loader' is your dataloader

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion_mse(outputs, labels)

        # output_aneurysm = torch.mean(outputs, dim=1)
        # loss_aneurysm = criterion_mse(output_aneurysm, labels[:,0])



        print(f"The loss at iteration {i} is {loss}")
        # Backward and optimize
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(),f'model_epoch_{epoch}.pth')

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to track the gradients
        total_val_loss = 0
        val_losses=[]
        for i, (patient_ids, images, labels) in enumerate(val_loader):  # Assuming 'val_loader' is your dataloader

            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)

            val_loss = criterion_mse(outputs, labels)
            total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f'Validation loss after epoch {epoch}: {avg_val_loss}')


cuda
Loaded checkpoint: /content/model_epoch_9.pth
The loss at iteration 0 is 0.03378419950604439
The loss at iteration 1 is 0.030988387763500214
The loss at iteration 2 is 0.022668303921818733
The loss at iteration 3 is 0.02312035858631134
The loss at iteration 4 is 0.035308632999658585
The loss at iteration 5 is 0.017765140160918236
The loss at iteration 6 is 0.022183988243341446
The loss at iteration 7 is 0.02826990745961666
The loss at iteration 8 is 0.09345275908708572
The loss at iteration 9 is 0.02255457080900669
The loss at iteration 10 is 0.017188560217618942
The loss at iteration 11 is 0.03122605010867119
The loss at iteration 12 is 0.03325493633747101
The loss at iteration 13 is 0.023407302796840668
The loss at iteration 14 is 0.044714804738759995
The loss at iteration 15 is 0.02056965045630932
The loss at iteration 16 is 0.02575160749256611
The loss at iteration 17 is 0.02505282312631607
The loss at iteration 18 is 0.017014790326356888
The loss at iteration 19 is 0.02016861

In [53]:
# Create a pandas dataframe to store the results
results = pd.DataFrame(columns=["Index", "Aneurysm"])
groundtruth = pd.DataFrame(columns=["Index", "Aneurysm"])

# Path to the saved model
checkpoint_path = "/content/model_epoch_9.pth"  # Replace this with your path

# Load the model weights
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f"Loaded checkpoint: {checkpoint_path}")
else:
    print("No checkpoint found at: "+checkpoint_path)
# No need to track the gradients
with torch.no_grad():
    for i, (patient_ids, images, labels) in enumerate(test_loader):


        labels = labels.cpu()
        labels_df = pd.DataFrame(labels.numpy())
        # Check if there are any 1s in the labels vector
        aneurysm_exists_gt = (torch.sum(labels, dim=1) > 0).float().cpu().numpy()
        # Append batch groundtruth to the results
        batch_groundtruth = pd.DataFrame({
            "Index": patient_ids.cpu().numpy(),
            "Aneurysm": aneurysm_exists_gt
        })

        # Concatenate batch_groundtruth with labels_df
        batch_groundtruth = pd.concat([batch_groundtruth, labels_df], axis=1)

        groundtruth = pd.concat([groundtruth, batch_groundtruth])


        images = images.to(device)
        # Forward pass
        outputs = model(images)
        # output_aneurysm=torch.mean(outputs, dim=1).float().cpu().numpy()
        output_aneurysm = torch.min(outputs, dim=1)[0].cpu().numpy()
        ########################
        # TODO : Among the 21 real values from the 'output' vector, detect outliers and change them into 1, and 0 otherwise.

        threshold = 0.55
        predicted_labels = (outputs > threshold).float().cpu().numpy()

          # z_scores = torch.abs((outputs - outputs.mean(axis=0)) / (outputs.std(axis=0)+eps))

          # z_scores =torch.nan_to_num(z_scores)
          # print(z_scores)
          # outputs = (z_scores > 1).float()
        ########################

        output_df = pd.DataFrame(predicted_labels)
        # Check if there are any 1s in the output vector


        # Append batch predictions to the results
        batch_results = pd.DataFrame({
            "Index": patient_ids.cpu().numpy(),
            "Aneurysm": output_aneurysm
        })

         # Concatenate batch_results with output_df
        batch_results = pd.concat([batch_results, output_df], axis=1)

        results = pd.concat([results, batch_results])

# Reset index
groundtruth.reset_index(drop=True, inplace=True)

# Save the groundtruth to a CSV file
groundtruth.to_csv('/content/test_set/groundtruth.csv', index=False)

# Reset index
results.reset_index(drop=True, inplace=True)

# Save the results to a CSV file
results.to_csv('/content/test_set/output.csv', index=False)


Loaded checkpoint: /content/model_epoch_9.pth


In [54]:
pd.read_csv('/content/test_set/output.csv')

,Index,Aneurysm,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,2312,0.010516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2156,0.010516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2361,0.010098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2583,0.009777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1399,0.009812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1412,0.009781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110,2567,0.009797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,2335,0.010516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,2522,0.009782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
!python test_set/evaluate.py


AUROC of the provided model
0.5711145996860282
Accuracy for locations
0.9724310776942355
